In [17]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re

In [18]:
df= pd.read_csv('latest_preprocessed_data.csv')

In [19]:
#removing stop words
def remove_stopwords (text):
    new_text = []
    for word in text.split():
        if word in stopwords.words('english'): new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

a=df['trans_review'].apply(remove_stopwords)

In [20]:
a[6]

'good surprise   power   processor   monster  power crucial point  heats  lot    put  aio  420mm  correctly cool  processor second point  consumes  lot  always   possibility  paying  maximum  125 watts   bios  deal   loss  4 8 perf honestly     gamer     need raw power turn   i5   make  trick'

In [21]:
#handling emojis , romving them 
import re
def remove_emoji(text):
    emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F" # emoticons
    u"\U0001F300-\U0001F5FF" # symbols & pictographs
    u"\U0001F680-\U0001F6FF" # transport & map symbols
    u"\U0001F1E0-\U0001F1FF" # flags (10S)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
a=a.apply(remove_emoji)

In [22]:
#removing the words which have numbers in them
def remove_words_with_numbers(input_string):
    # Split the input string into words
    words = input_string.split()

    # Filter out words containing numbers
    filtered_words = [word for word in words if not re.search(r'\d', word)]

    # Join the filtered words back into a string
    result = ' '.join(filtered_words)
    return result
a=a.apply(remove_words_with_numbers)
a

0       upgraded super happy performance able achieve ...
1       liked price also liked performance rather purc...
2       good processor gives able play perform office ...
3                       early anything expected satisfied
4                                  good decision computer
                              ...                        
1473    overall intel great work station however serie...
1474    written couple dozen reviews amazon second neg...
1475                             great cpu gaming regrets
1476                                                great
1477    system booted fine ram sticks installed appear...
Name: trans_review, Length: 1478, dtype: object

In [39]:
#tokenizing the text then applying lemmatization
from nltk.stem import WordNetLemmatizer
wordnetlemmatizer =WordNetLemmatizer()
def tokenize_lemmatize(text):
    tokenized_list = text.split(" ")
    lemmatized_list = [wordnetlemmatizer.lemmatize(word) for word in tokenized_list]
    return lemmatized_list
b=a.apply(tokenize_lemmatize)


In [40]:
#removing all the words which has its lenght less than 3 and more than 20
def filter_words(word_list):
    word_list= [word for word in word_list if 3 <= len(word) <= 20]
    return  ' '.join(word_list)
b=b.apply(filter_words)
b

0       upgraded super happy performance able achieve ...
1       liked price also liked performance rather purc...
2       good processor give able play perform office w...
3                       early anything expected satisfied
4                                  good decision computer
                              ...                        
1473    overall intel great work station however serie...
1474    written couple dozen review amazon second nega...
1475                              great cpu gaming regret
1476                                                great
1477    system booted fine ram stick installed appeare...
Name: trans_review, Length: 1478, dtype: object

In [43]:
#k means clustering to form clusters of the review sentiments
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
documents = b.values.astype("U")
vectorizer = TfidfVectorizer(stop_words='english')
features = vectorizer.fit_transform(documents) #extracting features from the data

In [44]:
features

<1478x4475 sparse matrix of type '<class 'numpy.float64'>'
	with 28481 stored elements in Compressed Sparse Row format>

In [48]:
k=3
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(features)

KMeans(max_iter=100, n_clusters=3, n_init=1)

In [56]:
cluster_data_labels=pd.Series(model.labels_)
cluster_data_labels

0       2
1       1
2       1
3       1
4       0
       ..
1473    2
1474    1
1475    2
1476    1
1477    2
Length: 1478, dtype: int32

In [57]:
df['review_sentiment_lables']=cluster_data_labels

In [69]:
#checking 5 reviews for each label to see if the classification makes sense
df[df['review_sentiment_lables']== 2].iloc[0:5,5]

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [70]:
df[df['review_sentiment_lables']== 1].iloc[0:5,5]

1                                                                                                                                                                                                                                                                                                   i don t liked the price of it   i also don t liked the performance of it   rather you can purchase i9 13th generation
2                                                                                                                                                                                                                                                                                                                         good processor that gives you to be able to play and perform office work  video rendering  etc 
3                                                                                                                                                                                   

In [72]:
df[df['review_sentiment_lables']== 0].iloc[0:5,5]

4                             good decision for my computer
16     very good processor  but needs a good refrigeration 
30                                               very good 
91                                                     good
104                       very good price performance ratio
Name: trans_review, dtype: object

In [73]:
#k means clustering doesnt see to have worked  , the difference is not see with the classification, now trying with k = 2
k=2
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(features)
cluster_data_labels=pd.Series(model.labels_)
cluster_data_labels

0       1
1       1
2       1
3       1
4       0
       ..
1473    1
1474    1
1475    1
1476    1
1477    1
Length: 1478, dtype: int32

In [74]:
df['review_sentiment_lables_k2']=cluster_data_labels

In [76]:
#checking 5 reviews for each label to see if the classification makes sense
df[df['review_sentiment_lables_k2']== 1].iloc[0:5,5]

0                                                                                                                                  just upgraded from i5 9400f to i9 14900k and super happy with the performance that i am able to achieve  it simply works buttery smooth  large applications open like notepad apps   it heats a lot though so i went with msi coreliquid liquid cooling solution   if looking to upgrade from 13th gen then save money because the performance is identical  however if setting up new pc then definitely go with it   my setup   msi mpg z790 carbon wifi intel i9 14900k msi rtx 4070ti msi mag coreliquid liquid cooler samsung 980 pro   1 tb ripjaws 32gb ddr5  bonus tip  if u are struck setting up ur pc then go to msi support and they are super responsive to assist you  i got my wiring fixed in matter of minutes ona video call by their assistant 
1                                                                                                                                 

In [77]:
df[df['review_sentiment_lables_k2']== 0].iloc[0:5,5]

4                                              good decision for my computer
16                      very good processor  but needs a good refrigeration 
30                                                                very good 
91                                                                      good
92    good processor  have a background check about who is selling you this 
Name: trans_review, dtype: object

In [78]:
#The the clusters have formed based on the length of the text rathere than the sentiment, so this doesnt seem to be working



SyntaxError: invalid syntax (3872961742.py, line 1)